## Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries

In [ ]:
!pip install -q pyabsa
!pip install -q datasets
import pandas as pd
import json
import ast
from pyabsa import ATEPCCheckpointManager
from collections import Counter
import os
import re
import string
from datasets import load_dataset
from datasets import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen impo

[2024-08-20 11:18:17] (2.4.1.post1) PyABSA(2.4.1.post1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


## Path (You need to modify this section based on your directory)

In [ ]:
# Set all paths accordingly based on your files:

# # Dataset
# csv_file_path = 'Path to the Dataset file (.csv)'
# # Save .json
# json_file_path = 'Path to save the .json file'
# # Save Aspect
# aspect_path = 'Path to save the aspects (.csv)'
# # Aspect Column
# Items_Column = "All_unique_items"
# Frequencies_Column = "All_frequencies"
# # Save Result
# Result_file_path = 'Path to save final result of ABSA (.csv)'

## Data Preprocessing

In [ ]:
# Function to handle initial preprocessing (conversion, NaNs, duplicates)
def initial_preprocessing(df):
    # Convert all entries to strings
    df['tweet.full_text'] = df['tweet.full_text'].astype(str)

    # Replace NaNs with empty strings
    df['tweet.full_text'] = df['tweet.full_text'].fillna('')

    # Remove duplicates
    df = df.drop_duplicates(subset=['tweet.full_text'])

    return df

# Function to handle text cleaning (lowercase, URLs, mentions, hashtags)
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#', '', text)

    cleaned_text = text

    return cleaned_text

# Function to filter short reviews
def filter_short_reviews(df, min_words=5, min_chars=50):
    # Filter out reviews with less than min_words words or less than min_chars characters
    df = df[df['cleaned_text'].apply(lambda x: len(x.split()) >= min_words and len(x) >= min_chars)]
    return df

# Function to preprocess the entire dataframe
def preprocess_dataframe(df):
    # Apply initial preprocessing
    df = initial_preprocessing(df)

    # Apply text cleaning
    df['cleaned_text'] = df['tweet.full_text'].apply(clean_text)

    # Filter short reviews
    df = filter_short_reviews(df)

    return df

In [ ]:
# Read the .csv file
df = pd.read_csv(csv_file_path)

# Apply preprocessing function
df = preprocess_dataframe(df)

# Extract the cleaned "tweet.full_text" column
reviews = df['cleaned_text'].tolist()

# Set batch size to the number of all data
batch_size = len(reviews)

In [ ]:
len(reviews)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


34002

## Load Model

In [ ]:
# Load the aspect extractor model
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english', auto_device=True) # False means load model on CPU

[2024-08-20 11:18:36] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-20 11:18:36] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-20 11:18:36] (2.4.1.post1) Downloading checkpoint:english 
[2024-08-20 11:18:36] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets


Find zipped checkpoint: ./checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43.zip, unzipping


<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


Done.
[2024-08-20 11:19:14] (2.4.1.post1) If the auto-downloading failed, please download it via browser: https://huggingface.co/spaces/yangheng/PyABSA/resolve/main/checkpoints/English/ATEPC/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43.zip 
[2024-08-20 11:19:15] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2024-08-20 11:19:15] (2.4.1.post1) config: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.config
[2024-08-20 11:19:15] (2.4.1.post1) state_dict: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.state_dict
[2024-08-20 11:19:15] (2.4.1.post1) model: None
[2024-08-20 11:19:15] (2.4.1.post1) tokenizer: checkpoints/ATEPC_ENGLISH_CHECKPOINT/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43/fast_lcf_atepc.tokenizer
[2024-08-20 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Performing ABSA

In [ ]:
# Function to process sentences in a single batch
def process_in_single_batch(sentences):
    result = aspect_extractor.extract_aspect(inference_source=sentences, pred_sentiment=True)
    return result

In [ ]:
# Process the reviews in a single batch
atepc_result = process_in_single_batch(reviews)

preparing apc inference dataloader: 100%|██████████| 48425/48425 [02:31<00:00, 319.35it/s]
/usr/local/lib/python3.10/dist-packages/pyabsa/tasks/AspectTermExtraction/prediction/aspect_extractor.py:593: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  lcf_cdm_vec = torch.tensor(
classifying aspect sentiments: 100%|██████████| 1514/1514 [11:42<00:00,  2.15it/s]


Streaming output truncated to the last 5000 lines.
[2024-08-20 11:43:34] (2.4.1.post1) Example 30971: i forgot to mention this . when you go , don ' t forget to bring a larger tote bag or similar bag to use when you buy doujinshi or small items , in addition to your regular bag . * it can be a pain if you end up with too many things lol * ★ in the end , if you don ' t find anything that you think is " nice , " then you ' re lucky . if you accidentally buy too many doujinshi , it will take up the <space:Negative Confidence:0.9807> of 2 - 4 laptops lol
[2024-08-20 11:43:34] (2.4.1.post1) Example 30972: autumn is full of materialistic desires . 
 
 ・ laptops 
 
 ・ tablets 
 
 ・ clothes , shoes , bags 
 
 ・ necklaces , bracelets 
 
 ・ perfume 
 
 and i ' <m also:Positive Confidence:0.9142> full of the desire to travel . 
 
 ・ yaeyama 
 
 ・ lake kawaguchi 
 
 ・ yaeyama 
 
 ・ yaeyama 
 
 ・ yaeyama
[2024-08-20 11:43:34] (2.4.1.post1) Example 30973:   ( 👼 top quality 🧚 💍 happiness magic 🧙 ‍ ♀ 

## Aspect Extraction

In [ ]:
# Extract the "aspect" values from the atepc_result variable and put them into a list
aspect_list = []
for result in atepc_result:
    aspect_list.extend(result["aspect"])

In [ ]:
# Sample list with some repeated items
original_list = aspect_list

# Extract unique items and their frequencies
item_counter = Counter(original_list)

# Create a new list with tuples (item, frequency), sorted by frequency in descending order
unique_items_with_frequency = sorted(item_counter.items(), key=lambda x: x[1], reverse=True)

# Separate the unique items and their frequencies
unique_items = [item for item, count in unique_items_with_frequency]
frequencies = [count for item, count in unique_items_with_frequency]

In [ ]:
# Print the results
print("Unique Items:", unique_items)
print("Frequencies:", frequencies)

## Save .json

In [ ]:
# Save the results to a JSON file
with open(json_file_path, 'w') as f:
    json.dump(atepc_result, f, indent=4)

## Convert .json to csv

In [ ]:
# Read the JSON file
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv(Result_file_path, index=False)

## Save Aspect.csv

In [ ]:
df = pd.read_csv(aspect_path)

# Add the new columns
df[Items_Column] = unique_items
df[Frequencies_Column] = frequencies

# Save the updated DataFrame back to the CSV file
df.to_csv(aspect_path, index=False)

print(f"Data has been successfully added to {aspect_path}")